In [1]:
!pip install transformers torch

In [2]:
from google.colab import files
from transformers import RobertaTokenizer, RobertaModel
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
import pickle

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('distilroberta-base')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
#Téléchargement des fichiers du colab
uploaded = files.upload()

Saving Compilation_from_Kaggle_IA_Human.csv to Compilation_from_Kaggle_IA_Human (1).csv


In [5]:
df = pd.read_csv("Compilation_from_Kaggle_IA_Human.csv")

In [6]:
texts = df["text"].tolist()
labels = df["label"].tolist()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [8]:
def get_embeddings_batch(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, max_length=512, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.extend(batch_embeddings)
    return embeddings

In [ ]:
X_train_embeddings = get_embeddings_batch(X_train, batch_size=32)

In [ ]:
X_test_embeddings = get_embeddings_batch(X_test, batch_size=32)

In [ ]:
X_train_embeddings = np.array(X_train_embeddings)
X_test_embeddings = np.array(X_test_embeddings)

In [ ]:
np.save("X_train_embeddings.npy", X_train_embeddings)
np.save("X_test_embeddings.npy", X_test_embeddings)

files.download("X_train_embeddings.npy")
files.download("X_test_embeddings.npy")

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_embeddings, y_train)

In [ ]:
y_pred = lr.predict(X_test_embeddings)

In [ ]:
print("Accuracy sur le train: ", accuracy_score(y_train, lr.predict(X_train_embeddings)))
print("Accuracy sur le test: ", accuracy_score(y_test, y_pred))
print("F1 score sur le test: ", f1_score(y_test, y_pred))
print("Precision sur le test: ", precision_score(y_test, y_pred))
print("Recall sur le test: ", recall_score(y_test, y_pred))
print("\nMatrice de confusion:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
with open('logistic_model.pkl', 'wb') as f:
  pickle.dump(lr, f)

files.download("logistic_model.pkl")